In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import time
import copy
import datetime
now = datetime.datetime.now()
from importlib import reload
import models
from models.y2014 import VGG
reload(models.y2014.VGG)

<module 'models.y2014.VGG' from 'C:\\Users\\siddi\\Documents\\GitHub\\Convolutional-Architecture-Timeline\\models\\y2014\\VGG.py'>

In [2]:
import torch 
from torch import nn
from torch.optim import Adam,RMSprop,SGD, lr_scheduler
from torch.autograd import backward
from torchvision.datasets import CIFAR100, CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torchvision
print("Runinng on :", device,"\nPyTorch Verison :",torch.__version__)

Runinng on : cuda:0 
PyTorch Verison : 1.5.0


In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = CIFAR10("./data/",train=True,transform = transform,download=True)
test_dataset = CIFAR10("./data/",train=False, transform = transform,download=True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
writer = SummaryWriter("./runs/"+now.strftime("%Y%m%d-%H%M%S")+"/cifar10_experiment")
batch_size = 4
train_dataloader = DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [5]:
print("Train Size",len(train_dataloader))
print("Test Size",len(test_dataloader))
dataloaders = {"train":train_dataloader, "val":test_dataloader}
dataset_sizes = {"train":len(train_dataset), "val":len(test_dataset)}

Train Size 12500
Test Size 2500


In [6]:
model = VGG.VGG()
model = model.to(device)

pytorch_total_params = sum(p.numel() for p in model.parameters())
print("Total Model Parameters:",pytorch_total_params)
pytorch_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total Trainable Parameters",pytorch_trainable_params)

images,labels = iter(train_dataloader).next()
img_grid = torchvision.utils.make_grid(images)
writer.add_image('One Batch of Images - CIFAR10'+str(labels), img_grid)

writer.add_graph(model,  torch.rand((3,32,32)).unsqueeze(0).cuda())
writer.close()

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

Total Model Parameters: 4330122
Total Trainable Parameters 4330122


In [8]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 100)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            print(datetime.datetime.now(),"\n")
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            count = 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                count = count +1 
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item()
                running_corrects += torch.sum(preds == labels.data)

                if count%100 == 0:
                    print("-"*40)
                    print("STEP : ",count,"/",len(dataloaders[phase]))
                    print('ACC : {:.4f}'.format(running_corrects.double()/(count*batch_size)), phase.upper(),"LOSS: {:.4f}".format(running_loss / count))
                    if phase == "train": writer.add_scalar('training loss',running_loss / count , epoch * len(train_dataloader) + count)
                    if phase == "train": writer.add_scalar('training Accuracy',running_corrects.double() / (count*batch_size) , epoch * len(train_dataloader) + count)
                    if phase == "test": writer.add_scalar('testing loss',running_loss / count , epoch * len(test_dataloader) + count)
                    if phase == "test": writer.add_scalar('testing Accuracy',running_corrects.double() / (count*batch_size) , epoch * len(test_dataloader) + count)
                    if phase == "train": writer.add_histogram("FC1 - Weights",model.fc1.weight.cpu().detach().numpy(),count)
                    if phase == "train": writer.add_histogram("Conv1 - Weights",model.conv_1.weight.cpu().detach().numpy(),count)
                    if phase == "train": writer.add_histogram("Conv2 - Weights",model.conv_2.weight.cpu().detach().numpy(),count)
                    if phase == "train": writer.add_histogram("Conv3 - Weights",model.conv_3.weight.cpu().detach().numpy(),count)
                    if phase == "train": writer.add_histogram("Conv4 - Weights",model.conv_4.weight.cpu().detach().numpy(),count)
                    writer.flush()
                    
            if phase == 'train':
                scheduler.step()
                

            epoch_loss = running_loss / len(dataloaders[phase])
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print("-"*100)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print("-"*100)
    print("-"*100)
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [9]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=30)

Epoch 0/29
----------------------------------------------------------------------------------------------------
2020-06-13 23:13:24.884814 

----------------------------------------
STEP :  100 / 12500
ACC : 0.1100 TRAIN LOSS: 2.3123
----------------------------------------
STEP :  200 / 12500
ACC : 0.1325 TRAIN LOSS: 2.2511
----------------------------------------
STEP :  300 / 12500
ACC : 0.1783 TRAIN LOSS: 2.1630
----------------------------------------
STEP :  400 / 12500
ACC : 0.2006 TRAIN LOSS: 2.1068
----------------------------------------
STEP :  500 / 12500
ACC : 0.2140 TRAIN LOSS: 2.0697
----------------------------------------
STEP :  600 / 12500
ACC : 0.2283 TRAIN LOSS: 2.0433
----------------------------------------
STEP :  700 / 12500
ACC : 0.2418 TRAIN LOSS: 2.0126
----------------------------------------
STEP :  800 / 12500
ACC : 0.2538 TRAIN LOSS: 1.9849
----------------------------------------
STEP :  900 / 12500
ACC : 0.2597 TRAIN LOSS: 1.9721
----------------------

KeyboardInterrupt: 

In [ ]:
model.fc1.weight

In [7]:
model

VGG(
  (conv_1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout1): Dropout(p=0.2, inplace=False)
  (conv_3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout2): Dropout(p=0.2, inplace=False)
  (fc1): Linear(in_features=8192, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)